In [ ]:
print("Hello World!")

In [1]:

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
# Ignore all FutureWarning warnings that might flood the console log
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import os
import pandas as pd

# from encryption import encode, decode

import time
from utils import create_patches_v3, create_binary_mask, create_patches_v2, create_patches_v1, create_patches_coordinates, create_patches_coordinates_v1
from scipy.stats.contingency import crosstab
from sklearn.metrics import mutual_info_score, normalized_mutual_info_score, adjusted_mutual_info_score

# Loading directory
wsi_dir = "/root/code2024/dataset/1WSI/data/"
patch_size = 224
cpu_core = 12

# Saving directory
save_dir = wsi_dir

downsize = patch_size
patch_extraction_size = patch_size
mask_overlap = 80.0
batch_size = 64
cpu_workers = cpu_core
use_prob_threshold = 0.8 # None  # whether to give final prediction {0,1} based on certain probability

# torch.manual_seed(250)
print('--------------Start--------------')
# read the files
wsi_files = os.listdir(wsi_dir)
# get all files except temp directory containing patches
wsi_files = [f for f in wsi_files if f.endswith("svs") or f.endswith("ndpi") or f.endswith("mrxs")]

print(f"Total files in {wsi_dir} directory are {len(wsi_files)}")

path = os.path.join(wsi_dir, "cnn_ensemble_updated")
if not os.path.exists(path):
    os.mkdir(path)

lst_patching_time = []

stat_NMI_graph = [] 
stat_NMI_base = []
coords = []
new_file_paths = []
# start patching process over a certain WSI
for i, f in enumerate(wsi_files):

    st_binarymask = time.time()
    new_file_path = os.path.join(path, f.split(".")[0])
    print(new_file_path)

    # just take the name not extension
    if not os.path.exists(new_file_path):
        os.mkdir(new_file_path)
        
    # find binary mask to locate tissue on WSI            
    w, h = create_binary_mask(wsi_dir, f, new_file_path, downsize=patch_size)

    # print(w/224, h/224)
    et_binarymask = time.time()

    binarymask_processtime = et_binarymask - st_binarymask
    print(f"Time spent on create_binary_mask is {binarymask_processtime} for {f}")
        
    st_patching = time.time()
    patch_folder = os.path.join(new_file_path, "patches")
    
    if not os.path.exists(patch_folder):
    # # else:
        os.mkdir(patch_folder) 
    if os.path.exists(patch_folder):
    #     os.rmdir(patch_folder)
    # elif not os.path.exists(patch_folder):
    # # else:
    #     os.mkdir(patch_folder) 
        # coordinates, id_to_coordinates = create_patches_coordinates(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #单进程
        # coordinates, id_to_coordinates, patches = create_patches_v1(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #单进程
        
        # create_patches_v3(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
                    #    patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        
        # 1. create coordinates with multi-processing
        res = create_patches_coordinates_v1(wsi_dir, f, new_file_path, workers=cpu_workers, patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        
        # patches = create_patches_v2(wsi_dir, f, new_file_path, patch_folder, workers=cpu_workers,
            # patch_size=patch_extraction_size, mask_overlap=mask_overlap) #多进程
        # print(len(coordinates))
        # print(len(patches))
        # print(id_to_coordinates)

        # print("Coordinates Done!", len(res), res[:3])
        # coordinates = list(res.values())
        coordinates = res.values()
        ids = res.keys()
        # print("Coordinates Done!", len(res), len(coordinates), coordinates[:3])
        coords.append(coordinates)
        new_file_paths.append(new_file_path)
        

--------------Start--------------
Total files in /root/code2024/dataset/1WSI/data/ directory are 1
/root/code2024/dataset/1WSI/data/cnn_ensemble_updated/TCGA-E9-A1N3-01Z-00-DX1

##########################################
Creating basic binary masks for TCGA-E9-A1N3-01Z-00-DX1.608fb360-17bf-4f85-b090-b8fc867831a3.svs
Time taken for creating binary mask 0.02 seconds
Time spent on create_binary_mask is 0.01902151107788086 for TCGA-E9-A1N3-01Z-00-DX1.608fb360-17bf-4f85-b090-b8fc867831a3.svs
Create patches for TCGA-E9-A1N3-01Z-00-DX1.608fb360-17bf-4f85-b090-b8fc867831a3.svs, using 12 CPUs out of 12
Patches coordinates created for TCGA-E9-A1N3-01Z-00-DX1.608fb360-17bf-4f85-b090-b8fc867831a3.svs in 0.19 minutes.


In [2]:
import os
import fnmatch
def count_files_with_numeric_pattern(start_path, file_pattern):
    count = 0
    for root, dirs, files in os.walk(start_path):
        for file in files:
            if fnmatch.fnmatch(file, file_pattern):
                count += 1
                # print(os.path.join(root, file))  # 打印匹配的文件路径
    return count

In [3]:
print(new_file_paths)

['/root/code2024/dataset/1WSI/data/cnn_ensemble_updated/TCGA-E9-A1N3-01Z-00-DX1']


# Simulation Data Collection

## Graph Splitting based simulation

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import statistics 
import openpyxl


def kLargest(arr, k, flag):
	# Sort the given array arr in reverse order.
	arr.sort(reverse=flag)
	# Print the first kth largest elements
	for i in range(k):
		print(arr[i], end=" ")

# Benchmark data
# dir_path = "/Volumes/T7/Research/WSI preprocessing project/Distributed Image Processing/WSI/benchmarks/"


# time cost of using CNN workloads: q*F(No. of patches) = latencies; --> latencies/F(No.of patches) --> time_cost_base: q second per patch 
time_base = 0.0050
# print(trans_time)
# lantency = No.of patches x 1/throughput_T4
throughput_T4 = 200   
# time = []

for new_file_path in new_file_paths: 
    graph_split_file = f'{new_file_path}/graph_split1/'
    os.listdir(graph_split_file)
    # strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            # 'connected_sequential', 'saturation_largest_first']
    # strategies = ['saturation_largest_first']
    strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 'connected_sequential', 'saturation_largest_first']

    time_per_strategy = {}
    for my_strategy in strategies: 
        
        # d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)
        
        # 设置你的目标文件夹名称和文件模式
        graph_split_file = f'{new_file_path}/graph_split1/'
        file_pattern = f"*[0-9]{my_strategy}.csv"

        # 计算文件数量
        dataset_number = count_files_with_numeric_pattern(graph_split_file, file_pattern)

        exe_tmp = []
        datasetSize = []
        for j in range(dataset_number):
            # sub data set
            origfpath = f'{graph_split_file}/graph_patchesnames_{str(j)}{my_strategy}.csv'
            df = pd.read_csv(origfpath)
            data_no = len(df['PatchName'])
            exetime4data_no = data_no * time_base
            # transtime4data_no = data_no * 0.8/SpeedTest["Amsterdam, the Netherlands"]
            exe_tmp.append(exetime4data_no)
            datasetSize.append(data_no)
        print(my_strategy, exe_tmp)
        # time_per_strategy[my_strategy] = tmp
        # print('Time_per_strategy: ', time_per_strategy)
        # time = time_per_strategy['saturation_largest_first']
        # exe_time = exe_tmp

        # lantency = No.of patches x 1/throughput_T4
        throughput_T4 = 200   
        time = exe_tmp
        print(datasetSize)
        # unit price
        UP = {"Tesla T4 (gpu-e)": 0,
            # "Tesla T4 (gpu-f)": 0,
            # "RTX 3090 (TY)": 0,
            "RTX 3090 (UiS)": 0,
            "A100 80GB": 2.15, 
            "A100 40GB": 2.01, 
            "V100 IL": 0.81,
            "RTX A6000 IL": 1.27,
            "RTX A6000 NY": 1.27,
            "RTX A5000 IL": 0.78,
            "RTX A5000 NV": 0.78,
            "RTX A5000 NY": 0.78,
            "RTX A4000 NY": 0.63,
            "RTX A4000 IL": 0.63,
            "A40 IL": 1.27, 
            "A40 NY": 1.27, 
            "A40 NV": 1.27, 
            "QUADRO RTX 5000 IL": 0.59, 
            "QUADRO RTX 5000 NY": 0.59, 
            "QUADRO RTX 5000 NV": 0.59, 
            "QUADRO RTX 4000 IL": 0.28,
            "QUADRO RTX 4000 NY": 0.28,
            "QUADRO RTX 4000 NV": 0.28,
            "RTX 3060 Ti NO": 0.34,
            "RTX 3090 NO": 0.59,
            "RTX 3090 IS": 0.59,
            "RTX 3080 NO": 0.50,
            "RTX 3080 IS": 0.50,
            "V100 IN": 1.07
            }


        # location 
        location = {"Tesla T4 (gpu-e)": "Amsterdam, the Netherlands",
            # "Tesla T4 (gpu-f)": "Amsterdam, the Netherlands",
            # "RTX 3090 (TY)": "Valencia, Spain",
            "RTX 3090 (UiS)": "Stavanger, Norway",
            "A100 80GB": "Illinois, USA", 
            "A100 40GB": "Illinois, USA", 
            "V100 IL": "Illinois, USA",
            "RTX A6000 IL": "Illinois, USA",
            "RTX A6000 NY": "New York, USA",
            "RTX A5000 IL": "Illinois, USA",
            "RTX A5000 NV": "Nevada, USA",
            "RTX A5000 NY": "New York, USA",
            "RTX A4000 NY": "New York, USA",
            "RTX A4000 IL": "Illinois, USA",
            "A40 IL": "Illinois, USA", 
            "A40 NY": "New York, USA", 
            "A40 NV": "Nevada, USA", 
            "QUADRO RTX 5000 IL": "Illinois, USA", 
            "QUADRO RTX 5000 NY": "New York, USA", 
            "QUADRO RTX 5000 NV": "Nevada, USA", 
            "QUADRO RTX 4000 IL": "Illinois, USA",
            "QUADRO RTX 4000 NY": "New York, USA",
            "QUADRO RTX 4000 NV": "Nevada, USA",
            "RTX 3060 Ti NO": "Stavanger, Norway",
            "RTX 3090 NO": "Stavanger, Norway",
            "RTX 3090 IS": "Iceland",
            "RTX 3080 NO": "Stavanger, Norway",
            "RTX 3080 IS": "Iceland",
            "V100 IN": "India"
            }

        # bandwidth --> upload speed (Mbps),1 Mbps = 0.125 MB/s
        SpeedTest = {"Amsterdam, the Netherlands": 40.26, #https://www.speedtest.net/performance/netherlands/north-holland/amsterdam
                    "Stavanger, Norway": 94.94, #https://www.speedtest.net/performance/norway/rogaland/stavanger
                    # "Valencia, Spain": 137.81, #https://www.speedtest.net/performance/spain/valencian-community/valencia
                    "Illinois, USA": 22.22, #https://www.speedtest.net/performance/united-states/illinois
                    "New York, USA": 29.56, #https://www.speedtest.net/performance/united-states/new-york/new-york
                    "Nevada, USA": 31.05, #https://www.speedtest.net/performance/united-states/nevada
                    "Iceland": 67.65, #https://www.bandwidthplace.com/location/iceland/
                    "India": 86.05 #https://www.speedtest.net/performance/india/andaman-and-nicobar-islands/port-blair
                    }

        # print(trans_time)
        # lantency = No.of patches x 1/throughput_T4
        # throughput_T4 = 200   
        # time = []

        time = [n/throughput_T4 for n in datasetSize] 
        # 100KB/x Mbps = 0.1MB/x * 0.125 MB/s = 0.8/x second
        # transmission  = No.of patches * 100KB / speed(upload)    # Let 
        trans_time = {"Tesla T4 (gpu-e)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
            # "Tesla T4 (gpu-f)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
            # "RTX 3090 (TY)": [n*0.8/SpeedTest["Valencia, Spain"] for n in datasetSize],
            "RTX 3090 (UiS)": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "A100 80GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "A100 40GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "V100 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A6000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A6000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
            "RTX A5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "A40 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "A40 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
            "A40 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
            "QUADRO RTX 4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "QUADRO RTX 4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "QUADRO RTX 4000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
            "RTX 3060 Ti NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3090 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3090 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
            "RTX 3080 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3080 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
            "V100 IN": [n*0.8/SpeedTest["India"] for n in datasetSize]
            }


        # compute time
        T3090 = [round(i/2.23, 2) for i in time]
        compt_d = {"Tesla T4 (gpu-e)": time,
            # "Tesla T4 (gpu-f)": time,
            # "RTX 3090 (TY)": T3090,
            "RTX 3090 (UiS)": T3090,
            "A100 80GB": [round(i*1.55, 2) for i in T3090], 
            "A100 40GB": [round(i*1.55, 2) for i in T3090], 
            "V100 IL": [round(i*1.62, 2) for i in T3090],
            "RTX A6000 IL": [round(i*0.90, 2) for i in T3090],
            "RTX A6000 NY": [round(i*0.90, 2) for i in T3090],
            "RTX A5000 IL": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NV": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NY": [round(i*1.22, 2) for i in T3090],
            "RTX A4000 NY": [round(i*1.57, 2) for i in T3090],
            "RTX A4000 IL": [round(i*1.57, 2) for i in T3090],
            "A40 IL": [round(i*0.93, 2) for i in T3090], 
            "A40 NY": [round(i*0.93, 2) for i in T3090], 
            "A40 NV": [round(i*0.93, 2) for i in T3090], 
            "QUADRO RTX 5000 IL": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NY": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NV": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 4000 IL": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NY": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NV": [round(i*2.38, 2) for i in T3090],
            "RTX 3060 Ti NO": [round(i*1.74, 2) for i in T3090],
            "RTX 3090 NO": T3090,
            "RTX 3090 IS": T3090,
            "RTX 3080 NO": [round(i*1.14, 2) for i in T3090],
            "RTX 3080 IS": [round(i*1.14, 2) for i in T3090],
            "V100 IN": [round(i*1.62, 2) for i in T3090]
            }

        # tot_time = compt_time+trans_time
        computeDf = pd.DataFrame(data=compt_d)
        transDF = pd.DataFrame(data=trans_time)

        tottDf = computeDf+transDF
        costDf = tottDf * UP
        print(tottDf)

        print(costDf)
        matrix = tottDf.to_numpy()
        cmatrix = costDf.to_numpy()

largest_first [19.395, 19.455000000000002, 19.045, 18.34, 14.865, 8.9, 1.97, 0.04]
[3879, 3891, 3809, 3668, 2973, 1780, 394, 8]
   Tesla T4 (gpu-e)  RTX 3090 (UiS)   A100 80GB   A100 40GB     V100 IL  RTX A6000 IL  RTX A6000 NY  RTX A5000 IL  RTX A5000 NV  RTX A5000 NY  RTX A4000 NY  RTX A4000 IL      A40 IL      A40 NY      A40 NV  QUADRO RTX 5000 IL  QUADRO RTX 5000 NY  QUADRO RTX 5000 NV  QUADRO RTX 4000 IL  QUADRO RTX 4000 NY  QUADRO RTX 4000 NV  RTX 3060 Ti NO  RTX 3090 NO  RTX 3090 IS  RTX 3080 NO  RTX 3080 IS    V100 IN
0         96.473987       41.385907  153.137966  153.137966  153.747966    147.487966    112.809702    150.267966    110.552029    115.589702    118.639702    153.317966  147.747966  113.069702  108.032029          155.147966          120.469702          115.432029          160.367966          125.689702          120.652029       47.825907    41.385907    54.571397    42.605907    55.791397  50.152754
1         96.772437       41.507023  153.610009  153.610009  1

## Evenly Splitting based Simulation

In [4]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import statistics 
import openpyxl


def kLargest(arr, k, flag):
	# Sort the given array arr in reverse order.
	arr.sort(reverse=flag)
	# Print the first kth largest elements
	for i in range(k):
		print(arr[i], end=" ")

# Benchmark data
# dir_path = "/Volumes/T7/Research/WSI preprocessing project/Distributed Image Processing/WSI/benchmarks/"


# time cost of using CNN workloads: q*F(No. of patches) = latencies; --> latencies/F(No.of patches) --> time_cost_base: q second per patch 
time_base = 0.0050
# print(trans_time)
# lantency = No.of patches x 1/throughput_T4
throughput_T4 = 200   
# time = []

for new_file_path in new_file_paths: 
    dataset_numbers = [4,5,6,7,8]
    strategies = ['is_shuffled_True', 'is_shuffled_False']
    time_per_strategy = {}

    for my_strategy in strategies: 
       # 设置你的目标文件夹名称和文件模式
        for dataset_number in dataset_numbers:  
            even_split_file = f'{new_file_path}/even_split/{dataset_number}/'
            # file_pattern = f"*[0-9]{my_strategy}.csv"

            # 计算文件数量
            # dataset_number = count_files_with_numeric_pattern(even_split_file, file_pattern)

            exe_tmp = []
            datasetSize = []
            for j in range(dataset_number):
                # sub data set
                origfpath = f'{even_split_file}even_patchesnames_{str(j)}_{my_strategy}.csv'
                df = pd.read_csv(origfpath)
                data_no = len(df['PatchName'])
                exetime4data_no = data_no * time_base
                # transtime4data_no = data_no * 0.8/SpeedTest["Amsterdam, the Netherlands"]
                exe_tmp.append(exetime4data_no)
                datasetSize.append(data_no)
            print(my_strategy, dataset_number, exe_tmp)
            # time_per_strategy[my_strategy] = tmp
            # print('Time_per_strategy: ', time_per_strategy)
            # time = time_per_strategy['saturation_largest_first']
            # exe_time = exe_tmp

            # lantency = No.of patches x 1/throughput_T4
            throughput_T4 = 200   
            time = exe_tmp
            print(datasetSize)
            # unit price
            UP = {"Tesla T4 (gpu-e)": 0,
                # "Tesla T4 (gpu-f)": 0,
                # "RTX 3090 (TY)": 0,
                "RTX 3090 (UiS)": 0,
                "A100 80GB": 2.15, 
                "A100 40GB": 2.01, 
                "V100 IL": 0.81,
                "RTX A6000 IL": 1.27,
                "RTX A6000 NY": 1.27,
                "RTX A5000 IL": 0.78,
                "RTX A5000 NV": 0.78,
                "RTX A5000 NY": 0.78,
                "RTX A4000 NY": 0.63,
                "RTX A4000 IL": 0.63,
                "A40 IL": 1.27, 
                "A40 NY": 1.27, 
                "A40 NV": 1.27, 
                "QUADRO RTX 5000 IL": 0.59, 
                "QUADRO RTX 5000 NY": 0.59, 
                "QUADRO RTX 5000 NV": 0.59, 
                "QUADRO RTX 4000 IL": 0.28,
                "QUADRO RTX 4000 NY": 0.28,
                "QUADRO RTX 4000 NV": 0.28,
                "RTX 3060 Ti NO": 0.34,
                "RTX 3090 NO": 0.59,
                "RTX 3090 IS": 0.59,
                "RTX 3080 NO": 0.50,
                "RTX 3080 IS": 0.50,
                "V100 IN": 1.07
                }


            # location 
            location = {"Tesla T4 (gpu-e)": "Amsterdam, the Netherlands",
                # "Tesla T4 (gpu-f)": "Amsterdam, the Netherlands",
                # "RTX 3090 (TY)": "Valencia, Spain",
                "RTX 3090 (UiS)": "Stavanger, Norway",
                "A100 80GB": "Illinois, USA", 
                "A100 40GB": "Illinois, USA", 
                "V100 IL": "Illinois, USA",
                "RTX A6000 IL": "Illinois, USA",
                "RTX A6000 NY": "New York, USA",
                "RTX A5000 IL": "Illinois, USA",
                "RTX A5000 NV": "Nevada, USA",
                "RTX A5000 NY": "New York, USA",
                "RTX A4000 NY": "New York, USA",
                "RTX A4000 IL": "Illinois, USA",
                "A40 IL": "Illinois, USA", 
                "A40 NY": "New York, USA", 
                "A40 NV": "Nevada, USA", 
                "QUADRO RTX 5000 IL": "Illinois, USA", 
                "QUADRO RTX 5000 NY": "New York, USA", 
                "QUADRO RTX 5000 NV": "Nevada, USA", 
                "QUADRO RTX 4000 IL": "Illinois, USA",
                "QUADRO RTX 4000 NY": "New York, USA",
                "QUADRO RTX 4000 NV": "Nevada, USA",
                "RTX 3060 Ti NO": "Stavanger, Norway",
                "RTX 3090 NO": "Stavanger, Norway",
                "RTX 3090 IS": "Iceland",
                "RTX 3080 NO": "Stavanger, Norway",
                "RTX 3080 IS": "Iceland",
                "V100 IN": "India"
                }

            # bandwidth --> upload speed (Mbps),1 Mbps = 0.125 MB/s
            SpeedTest = {"Amsterdam, the Netherlands": 40.26, #https://www.speedtest.net/performance/netherlands/north-holland/amsterdam
                        "Stavanger, Norway": 94.94, #https://www.speedtest.net/performance/norway/rogaland/stavanger
                        # "Valencia, Spain": 137.81, #https://www.speedtest.net/performance/spain/valencian-community/valencia
                        "Illinois, USA": 22.22, #https://www.speedtest.net/performance/united-states/illinois
                        "New York, USA": 29.56, #https://www.speedtest.net/performance/united-states/new-york/new-york
                        "Nevada, USA": 31.05, #https://www.speedtest.net/performance/united-states/nevada
                        "Iceland": 67.65, #https://www.bandwidthplace.com/location/iceland/
                        "India": 86.05 #https://www.speedtest.net/performance/india/andaman-and-nicobar-islands/port-blair
                        }

            # print(trans_time)
            # lantency = No.of patches x 1/throughput_T4
            # throughput_T4 = 200   
            # time = []

            time = [n/throughput_T4 for n in datasetSize] 
            # 100KB/x Mbps = 0.1MB/x * 0.125 MB/s = 0.8/x second
            # transmission  = No.of patches * 100KB / speed(upload)    # Let 
            trans_time = {"Tesla T4 (gpu-e)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
                # "Tesla T4 (gpu-f)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
                # "RTX 3090 (TY)": [n*0.8/SpeedTest["Valencia, Spain"] for n in datasetSize],
                "RTX 3090 (UiS)": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "A100 80GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "A100 40GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "V100 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A6000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A6000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
                "RTX A5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "A40 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "A40 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
                "A40 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
                "QUADRO RTX 4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "QUADRO RTX 4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "QUADRO RTX 4000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
                "RTX 3060 Ti NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3090 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3090 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
                "RTX 3080 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3080 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
                "V100 IN": [n*0.8/SpeedTest["India"] for n in datasetSize]
                }


            # compute time
            T3090 = [round(i/2.23, 2) for i in time]
            compt_d = {"Tesla T4 (gpu-e)": time,
                # "Tesla T4 (gpu-f)": time,
                # "RTX 3090 (TY)": T3090,
                "RTX 3090 (UiS)": T3090,
                "A100 80GB": [round(i*1.55, 2) for i in T3090], 
                "A100 40GB": [round(i*1.55, 2) for i in T3090], 
                "V100 IL": [round(i*1.62, 2) for i in T3090],
                "RTX A6000 IL": [round(i*0.90, 2) for i in T3090],
                "RTX A6000 NY": [round(i*0.90, 2) for i in T3090],
                "RTX A5000 IL": [round(i*1.22, 2) for i in T3090],
                "RTX A5000 NV": [round(i*1.22, 2) for i in T3090],
                "RTX A5000 NY": [round(i*1.22, 2) for i in T3090],
                "RTX A4000 NY": [round(i*1.57, 2) for i in T3090],
                "RTX A4000 IL": [round(i*1.57, 2) for i in T3090],
                "A40 IL": [round(i*0.93, 2) for i in T3090], 
                "A40 NY": [round(i*0.93, 2) for i in T3090], 
                "A40 NV": [round(i*0.93, 2) for i in T3090], 
                "QUADRO RTX 5000 IL": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 5000 NY": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 5000 NV": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 4000 IL": [round(i*2.38, 2) for i in T3090],
                "QUADRO RTX 4000 NY": [round(i*2.38, 2) for i in T3090],
                "QUADRO RTX 4000 NV": [round(i*2.38, 2) for i in T3090],
                "RTX 3060 Ti NO": [round(i*1.74, 2) for i in T3090],
                "RTX 3090 NO": T3090,
                "RTX 3090 IS": T3090,
                "RTX 3080 NO": [round(i*1.14, 2) for i in T3090],
                "RTX 3080 IS": [round(i*1.14, 2) for i in T3090],
                "V100 IN": [round(i*1.62, 2) for i in T3090]
                }

            # tot_time = compt_time+trans_time
            computeDf = pd.DataFrame(data=compt_d)
            transDF = pd.DataFrame(data=trans_time)

            tottDf = computeDf+transDF
            costDf = tottDf * UP
            print(tottDf)

            print(costDf)
            matrix = tottDf.to_numpy()
            cmatrix = costDf.to_numpy()

is_shuffled_True 4 [25.505, 25.505, 25.5, 25.5]
[5101, 5101, 5100, 5100]
   Tesla T4 (gpu-e)  RTX 3090 (UiS)   A100 80GB   A100 40GB     V100 IL  RTX A6000 IL  RTX A6000 NY  RTX A5000 IL  RTX A5000 NV  RTX A5000 NY  RTX A4000 NY  RTX A4000 IL      A40 IL      A40 NY      A40 NV  QUADRO RTX 5000 IL  QUADRO RTX 5000 NY  QUADRO RTX 5000 NV  QUADRO RTX 4000 IL  QUADRO RTX 4000 NY  QUADRO RTX 4000 NV  RTX 3060 Ti NO  RTX 3090 NO  RTX 3090 IS  RTX 3080 NO  RTX 3080 IS    V100 IN
0        126.866153       54.422937  201.384365  201.384365  202.184365    193.954365    148.351421    197.614365    145.386731    152.011421    156.011421    201.614365  194.294365  148.691421  142.066731          204.014365          158.411421          151.786731          210.884365          165.281421          158.656731       62.892937    54.422937    71.762247    56.022937    73.362247  65.953591
1        126.866153       54.422937  201.384365  201.384365  202.184365    193.954365    148.351421    197.614365    

# $\epsilon$-约束法求解双目标优化问题： 

选择一个目标作为主要优化目标：我们可以选择最小化总成本作为主要优化目标。
将另一个目标通过ε约束引入：我们将最大时间作为约束条件引入，即在每次求解时，限制最大时间不超过某个ε值。
逐步调整ε值：通过改变ε值，我们可以得到一系列解，这些解展示了两个目标之间的权衡关系。
下面是实现ε-约束法的Python代码示例，使用PuLP库来解决这个线性规划问题。 

In [5]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import statistics 
import openpyxl
import pulp


def kLargest(arr, k, flag):
	# Sort the given array arr in reverse order.
	arr.sort(reverse=flag)
	# Print the first kth largest elements
	for i in range(k):
		print(arr[i], end=" ")

# Benchmark data
# dir_path = "/Volumes/T7/Research/WSI preprocessing project/Distributed Image Processing/WSI/benchmarks/"


# time cost of using CNN workloads: q*F(No. of patches) = latencies; --> latencies/F(No.of patches) --> time_cost_base: q second per patch 
time_base = 0.0050
# print(trans_time)
# lantency = No.of patches x 1/throughput_T4
throughput_T4 = 200   
# time = []

for new_file_path in new_file_paths: 
    graph_split_file = f'{new_file_path}/graph_split1/'
    os.listdir(graph_split_file)
    # strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            # 'connected_sequential', 'saturation_largest_first']
    # strategies = ['saturation_largest_first']
    strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 'connected_sequential', 'saturation_largest_first']

    time_per_strategy = {}
    for my_strategy in strategies: 
        
        # d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)
        
        # 设置你的目标文件夹名称和文件模式
        graph_split_file = f'{new_file_path}/graph_split1/'
        file_pattern = f"*[0-9]{my_strategy}.csv"

        # 计算文件数量
        dataset_number = count_files_with_numeric_pattern(graph_split_file, file_pattern)

        exe_tmp = []
        datasetSize = []
        for j in range(dataset_number):
            # sub data set
            origfpath = f'{graph_split_file}/graph_patchesnames_{str(j)}{my_strategy}.csv'
            df = pd.read_csv(origfpath)
            data_no = len(df['PatchName'])
            exetime4data_no = data_no * time_base
            
            exe_tmp.append(exetime4data_no)
            datasetSize.append(data_no)
        print(my_strategy, exe_tmp)
        # time_per_strategy[my_strategy] = tmp
        # print('Time_per_strategy: ', time_per_strategy)
        # time = time_per_strategy['saturation_largest_first']
        # exe_time = exe_tmp

        # lantency = No.of patches x 1/throughput_T4
        throughput_T4 = 200   
        time = exe_tmp
        print(datasetSize)
        # unit price
        UP = {"Tesla T4 (gpu-e)": 0,
            # "Tesla T4 (gpu-f)": 0,
            # "RTX 3090 (TY)": 0,
            "RTX 3090 (UiS)": 0,
            "A100 80GB": 2.15, 
            "A100 40GB": 2.01, 
            "V100 IL": 0.81,
            "RTX A6000 IL": 1.27,
            "RTX A6000 NY": 1.27,
            "RTX A5000 IL": 0.78,
            "RTX A5000 NV": 0.78,
            "RTX A5000 NY": 0.78,
            "RTX A4000 NY": 0.63,
            "RTX A4000 IL": 0.63,
            "A40 IL": 1.27, 
            "A40 NY": 1.27, 
            "A40 NV": 1.27, 
            "QUADRO RTX 5000 IL": 0.59, 
            "QUADRO RTX 5000 NY": 0.59, 
            "QUADRO RTX 5000 NV": 0.59, 
            "QUADRO RTX 4000 IL": 0.28,
            "QUADRO RTX 4000 NY": 0.28,
            "QUADRO RTX 4000 NV": 0.28,
            "RTX 3060 Ti NO": 0.34,
            "RTX 3090 NO": 0.59,
            "RTX 3090 IS": 0.59,
            "RTX 3080 NO": 0.50,
            "RTX 3080 IS": 0.50,
            "V100 IN": 1.07
            }


        # location 
        location = {"Tesla T4 (gpu-e)": "Amsterdam, the Netherlands",
            # "Tesla T4 (gpu-f)": "Amsterdam, the Netherlands",
            # "RTX 3090 (TY)": "Valencia, Spain",
            "RTX 3090 (UiS)": "Stavanger, Norway",
            "A100 80GB": "Illinois, USA", 
            "A100 40GB": "Illinois, USA", 
            "V100 IL": "Illinois, USA",
            "RTX A6000 IL": "Illinois, USA",
            "RTX A6000 NY": "New York, USA",
            "RTX A5000 IL": "Illinois, USA",
            "RTX A5000 NV": "Nevada, USA",
            "RTX A5000 NY": "New York, USA",
            "RTX A4000 NY": "New York, USA",
            "RTX A4000 IL": "Illinois, USA",
            "A40 IL": "Illinois, USA", 
            "A40 NY": "New York, USA", 
            "A40 NV": "Nevada, USA", 
            "QUADRO RTX 5000 IL": "Illinois, USA", 
            "QUADRO RTX 5000 NY": "New York, USA", 
            "QUADRO RTX 5000 NV": "Nevada, USA", 
            "QUADRO RTX 4000 IL": "Illinois, USA",
            "QUADRO RTX 4000 NY": "New York, USA",
            "QUADRO RTX 4000 NV": "Nevada, USA",
            "RTX 3060 Ti NO": "Stavanger, Norway",
            "RTX 3090 NO": "Stavanger, Norway",
            "RTX 3090 IS": "Iceland",
            "RTX 3080 NO": "Stavanger, Norway",
            "RTX 3080 IS": "Iceland",
            "V100 IN": "India"
            }

        # bandwidth --> upload speed (Mbps),1 Mbps = 0.125 MB/s
        SpeedTest = {"Amsterdam, the Netherlands": 40.26, #https://www.speedtest.net/performance/netherlands/north-holland/amsterdam
                    "Stavanger, Norway": 94.94, #https://www.speedtest.net/performance/norway/rogaland/stavanger
                    # "Valencia, Spain": 137.81, #https://www.speedtest.net/performance/spain/valencian-community/valencia
                    "Illinois, USA": 22.22, #https://www.speedtest.net/performance/united-states/illinois
                    "New York, USA": 29.56, #https://www.speedtest.net/performance/united-states/new-york/new-york
                    "Nevada, USA": 31.05, #https://www.speedtest.net/performance/united-states/nevada
                    "Iceland": 67.65, #https://www.bandwidthplace.com/location/iceland/
                    "India": 86.05 #https://www.speedtest.net/performance/india/andaman-and-nicobar-islands/port-blair
                    }

        # print(trans_time)
        # lantency = No.of patches x 1/throughput_T4
        # throughput_T4 = 200   
        # time = []

        time = [n/throughput_T4 for n in datasetSize] 
        # 100KB/x Mbps = 0.1MB/x * 0.125 MB/s = 0.8/x second
        # transmission  = No.of patches * 100KB / speed(upload)    # Let 
        trans_time = {"Tesla T4 (gpu-e)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
            # "Tesla T4 (gpu-f)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
            # "RTX 3090 (TY)": [n*0.8/SpeedTest["Valencia, Spain"] for n in datasetSize],
            "RTX 3090 (UiS)": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "A100 80GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "A100 40GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "V100 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A6000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A6000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
            "RTX A5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "A40 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "A40 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
            "A40 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
            "QUADRO RTX 4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "QUADRO RTX 4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "QUADRO RTX 4000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
            "RTX 3060 Ti NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3090 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3090 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
            "RTX 3080 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3080 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
            "V100 IN": [n*0.8/SpeedTest["India"] for n in datasetSize]
            }


        # compute time
        T3090 = [round(i/2.23, 2) for i in time]
        compt_d = {"Tesla T4 (gpu-e)": time,
            # "Tesla T4 (gpu-f)": time,
            # "RTX 3090 (TY)": T3090,
            "RTX 3090 (UiS)": T3090,
            "A100 80GB": [round(i*1.55, 2) for i in T3090], 
            "A100 40GB": [round(i*1.55, 2) for i in T3090], 
            "V100 IL": [round(i*1.62, 2) for i in T3090],
            "RTX A6000 IL": [round(i*0.90, 2) for i in T3090],
            "RTX A6000 NY": [round(i*0.90, 2) for i in T3090],
            "RTX A5000 IL": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NV": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NY": [round(i*1.22, 2) for i in T3090],
            "RTX A4000 NY": [round(i*1.57, 2) for i in T3090],
            "RTX A4000 IL": [round(i*1.57, 2) for i in T3090],
            "A40 IL": [round(i*0.93, 2) for i in T3090], 
            "A40 NY": [round(i*0.93, 2) for i in T3090], 
            "A40 NV": [round(i*0.93, 2) for i in T3090], 
            "QUADRO RTX 5000 IL": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NY": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NV": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 4000 IL": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NY": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NV": [round(i*2.38, 2) for i in T3090],
            "RTX 3060 Ti NO": [round(i*1.74, 2) for i in T3090],
            "RTX 3090 NO": T3090,
            "RTX 3090 IS": T3090,
            "RTX 3080 NO": [round(i*1.14, 2) for i in T3090],
            "RTX 3080 IS": [round(i*1.14, 2) for i in T3090],
            "V100 IN": [round(i*1.62, 2) for i in T3090]
            }

        # tot_time = compt_time+trans_time
        computeDf = pd.DataFrame(data=compt_d)
        transDF = pd.DataFrame(data=trans_time)

        tottDf = computeDf+transDF
        costDf = tottDf * UP
        # print(df1)
        # matrix = tottDf.to_numpy()
        # cmatrix = costDf.to_numpy()

        
        time_matrix = tottDf.to_numpy()
        cost_matrix = costDf.to_numpy()
        print(tottDf)
        

        n, m = time_matrix.shape  # n行m列

        # 初始化一个空列表来存储每个ε值对应的解
        solutions = []

        # 定义不同的ε值（最大时间限制）
        epsilon_values = range(1, 60)  # 这个范围可以根据实际情况进行调整

        for epsilon in epsilon_values:
            # 创建问题实例
            prob = pulp.LpProblem("MinTotalCost_with_MaxTimeConstraint", pulp.LpMinimize)
            
            # 创建决策变量
            x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n) for j in range(m)), cat='Binary')

            # 目标函数：最小化总成本
            total_cost = pulp.lpSum(cost_matrix[i, j] * x[(i, j)] for i in range(n) for j in range(m))
            prob += total_cost

            # 约束条件
            for i in range(n):
                prob += pulp.lpSum(x[(i, j)] for j in range(m)) == 1  # 每行选择一个元素

            for j in range(m):
                prob += pulp.lpSum(x[(i, j)] for i in range(n)) <= 1  # 每列最多选择一个元素

            # 最大时间约束
            for i in range(n):
                for j in range(m):
                    prob += x[(i, j)] * time_matrix[i, j] <= epsilon

            # 求解问题
            status = prob.solve()

            # 如果有可行解，则保存结果
            if status == pulp.LpStatusOptimal:
                solution = {
                    'epsilon': epsilon,
                    'total_cost': pulp.value(total_cost),
                    'assignment': [(i, j) for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) == 1]
                }
                solutions.append(solution)

        print(my_strategy)
        # 打印解决方案
        for solution in solutions:
            print(f"当最大时间限制为 {solution['epsilon']} 时，最小总成本为 {solution['total_cost']}, 分配方案为 {solution['assignment']}")
        # print(solutions)
   

largest_first [19.395, 19.455000000000002, 19.045, 18.34, 14.865, 8.9, 1.97, 0.04]
[3879, 3891, 3809, 3668, 2973, 1780, 394, 8]
   Tesla T4 (gpu-e)  RTX 3090 (UiS)   A100 80GB   A100 40GB     V100 IL  RTX A6000 IL  RTX A6000 NY  RTX A5000 IL  RTX A5000 NV  RTX A5000 NY  RTX A4000 NY  RTX A4000 IL      A40 IL      A40 NY      A40 NV  QUADRO RTX 5000 IL  QUADRO RTX 5000 NY  QUADRO RTX 5000 NV  QUADRO RTX 4000 IL  QUADRO RTX 4000 NY  QUADRO RTX 4000 NV  RTX 3060 Ti NO  RTX 3090 NO  RTX 3090 IS  RTX 3080 NO  RTX 3080 IS    V100 IN
0         96.473987       41.385907  153.137966  153.137966  153.747966    147.487966    112.809702    150.267966    110.552029    115.589702    118.639702    153.317966  147.747966  113.069702  108.032029          155.147966          120.469702          115.432029          160.367966          125.689702          120.652029       47.825907    41.385907    54.571397    42.605907    55.791397  50.152754
1         96.772437       41.507023  153.610009  153.610009  1

In [ ]:
print(solutions)

# Minimize the maximum time under the total cost constriants
Graph-based splitting

Budget = 100
Budget = 120

In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import statistics 
import openpyxl
import pulp


def kLargest(arr, k, flag):
	# Sort the given array arr in reverse order.
	arr.sort(reverse=flag)
	# Print the first kth largest elements
	for i in range(k):
		print(arr[i], end=" ")

# Benchmark data
# dir_path = "/Volumes/T7/Research/WSI preprocessing project/Distributed Image Processing/WSI/benchmarks/"


# time cost of using CNN workloads: q*F(No. of patches) = latencies; --> latencies/F(No.of patches) --> time_cost_base: q second per patch 
time_base = 0.0050
# print(trans_time)
# lantency = No.of patches x 1/throughput_T4
throughput_T4 = 200   
# time = []

for new_file_path in new_file_paths: 
    graph_split_file = f'{new_file_path}/graph_split1/'
    os.listdir(graph_split_file)
    # strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 
            # 'connected_sequential', 'saturation_largest_first']
    # strategies = ['saturation_largest_first']
    strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 'connected_sequential', 'saturation_largest_first']

    time_per_strategy = {}
    solutions4strategies = {}
    for my_strategy in strategies: 
        
        # d, dataset_number = graph_coloring_based_splitting_visual1(graph_split_file, coordinates, patch_size, strategy=my_strategy)
        
        # 设置你的目标文件夹名称和文件模式
        graph_split_file = f'{new_file_path}/graph_split1/'
        file_pattern = f"*[0-9]{my_strategy}.csv"

        # 计算文件数量
        dataset_number = count_files_with_numeric_pattern(graph_split_file, file_pattern)

        exe_tmp = []
        datasetSize = []
        for j in range(dataset_number):
            # sub data set
            origfpath = f'{graph_split_file}/graph_patchesnames_{str(j)}{my_strategy}.csv'
            df = pd.read_csv(origfpath)
            data_no = len(df['PatchName'])
            exetime4data_no = data_no * time_base
            
            exe_tmp.append(exetime4data_no)
            datasetSize.append(data_no)
        print(my_strategy, exe_tmp)
        # time_per_strategy[my_strategy] = tmp
        # print('Time_per_strategy: ', time_per_strategy)
        # time = time_per_strategy['saturation_largest_first']
        # exe_time = exe_tmp

        # lantency = No.of patches x 1/throughput_T4
        throughput_T4 = 200   
        time = exe_tmp
        print(datasetSize)
        # unit price
        UP = {"Tesla T4 (gpu-e)": 0,
            # "Tesla T4 (gpu-f)": 0,
            # "RTX 3090 (TY)": 0,
            "RTX 3090 (UiS)": 0,
            "A100 80GB": 2.15, 
            "A100 40GB": 2.01, 
            "V100 IL": 0.81,
            "RTX A6000 IL": 1.27,
            "RTX A6000 NY": 1.27,
            "RTX A5000 IL": 0.78,
            "RTX A5000 NV": 0.78,
            "RTX A5000 NY": 0.78,
            "RTX A4000 NY": 0.63,
            "RTX A4000 IL": 0.63,
            "A40 IL": 1.27, 
            "A40 NY": 1.27, 
            "A40 NV": 1.27, 
            "QUADRO RTX 5000 IL": 0.59, 
            "QUADRO RTX 5000 NY": 0.59, 
            "QUADRO RTX 5000 NV": 0.59, 
            "QUADRO RTX 4000 IL": 0.28,
            "QUADRO RTX 4000 NY": 0.28,
            "QUADRO RTX 4000 NV": 0.28,
            "RTX 3060 Ti NO": 0.34,
            "RTX 3090 NO": 0.59,
            "RTX 3090 IS": 0.59,
            "RTX 3080 NO": 0.50,
            "RTX 3080 IS": 0.50,
            "V100 IN": 1.07
            }


        # location 
        location = {"Tesla T4 (gpu-e)": "Amsterdam, the Netherlands",
            # "Tesla T4 (gpu-f)": "Amsterdam, the Netherlands",
            # "RTX 3090 (TY)": "Valencia, Spain",
            "RTX 3090 (UiS)": "Stavanger, Norway",
            "A100 80GB": "Illinois, USA", 
            "A100 40GB": "Illinois, USA", 
            "V100 IL": "Illinois, USA",
            "RTX A6000 IL": "Illinois, USA",
            "RTX A6000 NY": "New York, USA",
            "RTX A5000 IL": "Illinois, USA",
            "RTX A5000 NV": "Nevada, USA",
            "RTX A5000 NY": "New York, USA",
            "RTX A4000 NY": "New York, USA",
            "RTX A4000 IL": "Illinois, USA",
            "A40 IL": "Illinois, USA", 
            "A40 NY": "New York, USA", 
            "A40 NV": "Nevada, USA", 
            "QUADRO RTX 5000 IL": "Illinois, USA", 
            "QUADRO RTX 5000 NY": "New York, USA", 
            "QUADRO RTX 5000 NV": "Nevada, USA", 
            "QUADRO RTX 4000 IL": "Illinois, USA",
            "QUADRO RTX 4000 NY": "New York, USA",
            "QUADRO RTX 4000 NV": "Nevada, USA",
            "RTX 3060 Ti NO": "Stavanger, Norway",
            "RTX 3090 NO": "Stavanger, Norway",
            "RTX 3090 IS": "Iceland",
            "RTX 3080 NO": "Stavanger, Norway",
            "RTX 3080 IS": "Iceland",
            "V100 IN": "India"
            }

        # bandwidth --> upload speed (Mbps),1 Mbps = 0.125 MB/s
        SpeedTest = {"Amsterdam, the Netherlands": 40.26, #https://www.speedtest.net/performance/netherlands/north-holland/amsterdam
                    "Stavanger, Norway": 94.94, #https://www.speedtest.net/performance/norway/rogaland/stavanger
                    # "Valencia, Spain": 137.81, #https://www.speedtest.net/performance/spain/valencian-community/valencia
                    "Illinois, USA": 22.22, #https://www.speedtest.net/performance/united-states/illinois
                    "New York, USA": 29.56, #https://www.speedtest.net/performance/united-states/new-york/new-york
                    "Nevada, USA": 31.05, #https://www.speedtest.net/performance/united-states/nevada
                    "Iceland": 67.65, #https://www.bandwidthplace.com/location/iceland/
                    "India": 86.05 #https://www.speedtest.net/performance/india/andaman-and-nicobar-islands/port-blair
                    }

        # print(trans_time)
        # lantency = No.of patches x 1/throughput_T4
        # throughput_T4 = 200   
        # time = []

        time = [n/throughput_T4 for n in datasetSize] 
        # 100KB/x Mbps = 0.1MB/x * 0.125 MB/s = 0.8/x second
        # transmission  = No.of patches * 100KB / speed(upload)    # Let 
        trans_time = {"Tesla T4 (gpu-e)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
            # "Tesla T4 (gpu-f)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
            # "RTX 3090 (TY)": [n*0.8/SpeedTest["Valencia, Spain"] for n in datasetSize],
            "RTX 3090 (UiS)": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "A100 80GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "A100 40GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "V100 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A6000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A6000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "RTX A5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
            "RTX A5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "RTX A4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "A40 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "A40 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
            "A40 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
            "QUADRO RTX 5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
            "QUADRO RTX 4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
            "QUADRO RTX 4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
            "QUADRO RTX 4000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
            "RTX 3060 Ti NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3090 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3090 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
            "RTX 3080 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
            "RTX 3080 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
            "V100 IN": [n*0.8/SpeedTest["India"] for n in datasetSize]
            }


        # compute time
        T3090 = [round(i/2.23, 2) for i in time]
        compt_d = {"Tesla T4 (gpu-e)": time,
            # "Tesla T4 (gpu-f)": time,
            # "RTX 3090 (TY)": T3090,
            "RTX 3090 (UiS)": T3090,
            "A100 80GB": [round(i*1.55, 2) for i in T3090], 
            "A100 40GB": [round(i*1.55, 2) for i in T3090], 
            "V100 IL": [round(i*1.62, 2) for i in T3090],
            "RTX A6000 IL": [round(i*0.90, 2) for i in T3090],
            "RTX A6000 NY": [round(i*0.90, 2) for i in T3090],
            "RTX A5000 IL": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NV": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NY": [round(i*1.22, 2) for i in T3090],
            "RTX A4000 NY": [round(i*1.57, 2) for i in T3090],
            "RTX A4000 IL": [round(i*1.57, 2) for i in T3090],
            "A40 IL": [round(i*0.93, 2) for i in T3090], 
            "A40 NY": [round(i*0.93, 2) for i in T3090], 
            "A40 NV": [round(i*0.93, 2) for i in T3090], 
            "QUADRO RTX 5000 IL": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NY": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NV": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 4000 IL": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NY": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NV": [round(i*2.38, 2) for i in T3090],
            "RTX 3060 Ti NO": [round(i*1.74, 2) for i in T3090],
            "RTX 3090 NO": T3090,
            "RTX 3090 IS": T3090,
            "RTX 3080 NO": [round(i*1.14, 2) for i in T3090],
            "RTX 3080 IS": [round(i*1.14, 2) for i in T3090],
            "V100 IN": [round(i*1.62, 2) for i in T3090]
            }

        # tot_time = compt_time+trans_time
        computeDf = pd.DataFrame(data=compt_d)
        transDF = pd.DataFrame(data=trans_time)

        tottDf = computeDf+transDF
        costDf = tottDf * UP
        # print(df1)
        # matrix = tottDf.to_numpy()
        # cmatrix = costDf.to_numpy()

        
        time_matrix = tottDf.to_numpy()
        cost_matrix = costDf.to_numpy()
        print(costDf)
        

        n, m = time_matrix.shape  # n行m列
        print(n, m)

        # 定义最大总成本ε
        # max_total_cost = 100  # 这个值根据实际情况调整
        # max_total_cost = 120  # 这个值根据实际情况调整
        max_total_cost = 80  # 这个值根据实际情况调整

        # 创建问题实例
        prob = pulp.LpProblem("MinimizeMaxTimeUnderCostConstraint", pulp.LpMinimize)

        # 定义决策变量
        x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n) for j in range(m)), cat='Binary')
        # 定义一个辅助变量来表示最大完成时间
        max_time = pulp.LpVariable("max_time", lowBound=0)

        # 目标函数：最小化最大完成时间
        prob += max_time

        # 约束条件
        # 每行恰好选择一个元素
        for i in range(n):
            prob += pulp.lpSum(x[(i, j)] for j in range(m)) == 1

        # 每列最多选择一个元素
        for j in range(m):
            prob += pulp.lpSum(x[(i, j)] for i in range(n)) <= 1

        # 总成本不超过ε
        prob += pulp.lpSum(x[(i, j)] * cost_matrix[i, j] for i in range(n) for j in range(m)) <= max_total_cost

        # 最大完成时间约束
        for i in range(n):
            for j in range(m):
                prob += x[(i, j)] * time_matrix[i, j] <= max_time

        # 求解问题
        prob.solve()

        # 检查是否有最优解
        if pulp.LpStatus[prob.status] == "Optimal":
            solution = [j for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) > 0.5]
            total_cost = sum(cost_matrix[i, solution[i]] for i in range(n))
            max_time_value = pulp.value(max_time)
            assignment = [(i, j) for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) == 1]
            print(f"Solution: {solution}, Total Cost: {total_cost}, Max Time: {max_time_value}, Assignment: {assignment}")
            solution_per_strategy = {'Total Cost': total_cost,
                                'Max Time': max_time_value,
                                'Solution': solution,
                                'assignment': assignment }
            solutions4strategies[my_strategy] = solution_per_strategy
        else:
            print("No optimal solution found.")
            continue 

        print(my_strategy) 
        
        # # 打印解决方案
        # for solution in solutions:
        #     print(f"当最大时间限制为 {solution['epsilon']} 时，最小总成本为 {solution['total_cost']}, 分配方案为 {solution['assignment']}")
        # # print(solutions)
   

        

largest_first [19.395, 19.455000000000002, 19.045, 18.34, 14.865, 8.9, 1.97, 0.04]
[3879, 3891, 3809, 3668, 2973, 1780, 394, 8]
   Tesla T4 (gpu-e)  RTX 3090 (UiS)   A100 80GB   A100 40GB     V100 IL  RTX A6000 IL  RTX A6000 NY  RTX A5000 IL  RTX A5000 NV  RTX A5000 NY  RTX A4000 NY  RTX A4000 IL      A40 IL      A40 NY      A40 NV  QUADRO RTX 5000 IL  QUADRO RTX 5000 NY  QUADRO RTX 5000 NV  QUADRO RTX 4000 IL  QUADRO RTX 4000 NY  QUADRO RTX 4000 NV  RTX 3060 Ti NO  RTX 3090 NO  RTX 3090 IS  RTX 3080 NO  RTX 3080 IS    V100 IN
0               0.0             0.0  329.246626  307.807311  124.535852    187.309717    143.268322    117.209013     86.230583     90.159968     74.743012     96.590318  187.639917  143.598522  137.200677           91.537300           71.077124           68.104897           44.903030           35.193117           33.782568       16.260808    24.417685    32.197124    21.302953    27.895698  53.663447
1               0.0             0.0  330.261519  308.756118  1

In [24]:
print(solutions4strategies)

{'largest_first': {'Total Cost': 77.80762828509576, 'Max Time': 73.941006, 'Solution': [21, 22, 24, 1, 0, 25, 23, 20], 'assignment': [(0, 21), (1, 22), (2, 24), (3, 1), (4, 0), (5, 25), (6, 23), (7, 20)]}, 'random_sequential': {'Total Cost': 76.90659448578192, 'Max Time': 74.985581, 'Solution': [1, 24, 22, 21, 0, 25, 23, 20], 'assignment': [(0, 1), (1, 24), (2, 22), (3, 21), (4, 0), (5, 25), (6, 23), (7, 20)]}, 'smallest_last': {'Total Cost': 79.71465649884136, 'Max Time': 60.541925, 'Solution': [24, 1, 22, 21, 0], 'assignment': [(0, 24), (1, 1), (2, 22), (3, 21), (4, 0)]}, 'independent_set': {'Total Cost': 78.78460644082051, 'Max Time': 57.4032, 'Solution': [22, 1, 24, 21, 0, 25, 23], 'assignment': [(0, 22), (1, 1), (2, 24), (3, 21), (4, 0), (5, 25), (6, 23)]}, 'connected_sequential': {'Total Cost': 79.73586276901693, 'Max Time': 59.860048, 'Solution': [22, 24, 1, 21, 0, 25], 'assignment': [(0, 22), (1, 24), (2, 1), (3, 21), (4, 0), (5, 25)]}, 'saturation_largest_first': {'Total Cost'

# Evenly Splitting Solutions
Budget=100

Budget=120

In [21]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import statistics 
import openpyxl

import pulp


def kLargest(arr, k, flag):
	# Sort the given array arr in reverse order.
	arr.sort(reverse=flag)
	# Print the first kth largest elements
	for i in range(k):
		print(arr[i], end=" ")

# Benchmark data
# dir_path = "/Volumes/T7/Research/WSI preprocessing project/Distributed Image Processing/WSI/benchmarks/"


# time cost of using CNN workloads: q*F(No. of patches) = latencies; --> latencies/F(No.of patches) --> time_cost_base: q second per patch 
time_base = 0.0050
# print(trans_time)
# lantency = No.of patches x 1/throughput_T4
throughput_T4 = 200   
# time = []

# Even_solutions4strategies= {}
for new_file_path in new_file_paths: 
    dataset_numbers = [4,5,6,7,8]
    strategies = ['is_shuffled_True', 'is_shuffled_False']
    time_per_strategy = {}
    Even_solutions4strategies = {}
    for my_strategy in strategies: 
       # 设置你的目标文件夹名称和文件模式
        for dataset_number in dataset_numbers:  
            even_split_file = f'{new_file_path}/even_split/{dataset_number}/'
            # file_pattern = f"*[0-9]{my_strategy}.csv"

            # 计算文件数量
            # dataset_number = count_files_with_numeric_pattern(even_split_file, file_pattern)

            exe_tmp = []
            datasetSize = []
            for j in range(dataset_number):
                # sub data set
                origfpath = f'{even_split_file}even_patchesnames_{str(j)}_{my_strategy}.csv'
                df = pd.read_csv(origfpath)
                data_no = len(df['PatchName'])
                exetime4data_no = data_no * time_base
                # transtime4data_no = data_no * 0.8/SpeedTest["Amsterdam, the Netherlands"]
                exe_tmp.append(exetime4data_no)
                datasetSize.append(data_no)
            print(my_strategy, dataset_number, exe_tmp)
            # time_per_strategy[my_strategy] = tmp
            # print('Time_per_strategy: ', time_per_strategy)
            # time = time_per_strategy['saturation_largest_first']
            # exe_time = exe_tmp

            # lantency = No.of patches x 1/throughput_T4
            throughput_T4 = 200   
            time = exe_tmp
            print(datasetSize)
            # unit price
            UP = {"Tesla T4 (gpu-e)": 0,
                # "Tesla T4 (gpu-f)": 0,
                # "RTX 3090 (TY)": 0,
                "RTX 3090 (UiS)": 0,
                "A100 80GB": 2.15, 
                "A100 40GB": 2.01, 
                "V100 IL": 0.81,
                "RTX A6000 IL": 1.27,
                "RTX A6000 NY": 1.27,
                "RTX A5000 IL": 0.78,
                "RTX A5000 NV": 0.78,
                "RTX A5000 NY": 0.78,
                "RTX A4000 NY": 0.63,
                "RTX A4000 IL": 0.63,
                "A40 IL": 1.27, 
                "A40 NY": 1.27, 
                "A40 NV": 1.27, 
                "QUADRO RTX 5000 IL": 0.59, 
                "QUADRO RTX 5000 NY": 0.59, 
                "QUADRO RTX 5000 NV": 0.59, 
                "QUADRO RTX 4000 IL": 0.28,
                "QUADRO RTX 4000 NY": 0.28,
                "QUADRO RTX 4000 NV": 0.28,
                "RTX 3060 Ti NO": 0.34,
                "RTX 3090 NO": 0.59,
                "RTX 3090 IS": 0.59,
                "RTX 3080 NO": 0.50,
                "RTX 3080 IS": 0.50,
                "V100 IN": 1.07
                }


            # location 
            location = {"Tesla T4 (gpu-e)": "Amsterdam, the Netherlands",
                # "Tesla T4 (gpu-f)": "Amsterdam, the Netherlands",
                # "RTX 3090 (TY)": "Valencia, Spain",
                "RTX 3090 (UiS)": "Stavanger, Norway",
                "A100 80GB": "Illinois, USA", 
                "A100 40GB": "Illinois, USA", 
                "V100 IL": "Illinois, USA",
                "RTX A6000 IL": "Illinois, USA",
                "RTX A6000 NY": "New York, USA",
                "RTX A5000 IL": "Illinois, USA",
                "RTX A5000 NV": "Nevada, USA",
                "RTX A5000 NY": "New York, USA",
                "RTX A4000 NY": "New York, USA",
                "RTX A4000 IL": "Illinois, USA",
                "A40 IL": "Illinois, USA", 
                "A40 NY": "New York, USA", 
                "A40 NV": "Nevada, USA", 
                "QUADRO RTX 5000 IL": "Illinois, USA", 
                "QUADRO RTX 5000 NY": "New York, USA", 
                "QUADRO RTX 5000 NV": "Nevada, USA", 
                "QUADRO RTX 4000 IL": "Illinois, USA",
                "QUADRO RTX 4000 NY": "New York, USA",
                "QUADRO RTX 4000 NV": "Nevada, USA",
                "RTX 3060 Ti NO": "Stavanger, Norway",
                "RTX 3090 NO": "Stavanger, Norway",
                "RTX 3090 IS": "Iceland",
                "RTX 3080 NO": "Stavanger, Norway",
                "RTX 3080 IS": "Iceland",
                "V100 IN": "India"
                }

            # bandwidth --> upload speed (Mbps),1 Mbps = 0.125 MB/s
            SpeedTest = {"Amsterdam, the Netherlands": 40.26, #https://www.speedtest.net/performance/netherlands/north-holland/amsterdam
                        "Stavanger, Norway": 94.94, #https://www.speedtest.net/performance/norway/rogaland/stavanger
                        # "Valencia, Spain": 137.81, #https://www.speedtest.net/performance/spain/valencian-community/valencia
                        "Illinois, USA": 22.22, #https://www.speedtest.net/performance/united-states/illinois
                        "New York, USA": 29.56, #https://www.speedtest.net/performance/united-states/new-york/new-york
                        "Nevada, USA": 31.05, #https://www.speedtest.net/performance/united-states/nevada
                        "Iceland": 67.65, #https://www.bandwidthplace.com/location/iceland/
                        "India": 86.05 #https://www.speedtest.net/performance/india/andaman-and-nicobar-islands/port-blair
                        }

            # print(trans_time)
            # lantency = No.of patches x 1/throughput_T4
            # throughput_T4 = 200   
            # time = []

            time = [n/throughput_T4 for n in datasetSize] 
            # 100KB/x Mbps = 0.1MB/x * 0.125 MB/s = 0.8/x second
            # transmission  = No.of patches * 100KB / speed(upload)    # Let 
            trans_time = {"Tesla T4 (gpu-e)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
                # "Tesla T4 (gpu-f)": [n*0.8/SpeedTest["Amsterdam, the Netherlands"] for n in datasetSize],
                # "RTX 3090 (TY)": [n*0.8/SpeedTest["Valencia, Spain"] for n in datasetSize],
                "RTX 3090 (UiS)": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "A100 80GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "A100 40GB": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "V100 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A6000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A6000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "RTX A5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
                "RTX A5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "RTX A4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "A40 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "A40 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
                "A40 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize], 
                "QUADRO RTX 5000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize], 
                "QUADRO RTX 4000 IL": [n*0.8/SpeedTest["Illinois, USA"] for n in datasetSize],
                "QUADRO RTX 4000 NY": [n*0.8/SpeedTest["New York, USA"] for n in datasetSize],
                "QUADRO RTX 4000 NV": [n*0.8/SpeedTest["Nevada, USA"] for n in datasetSize],
                "RTX 3060 Ti NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3090 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3090 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
                "RTX 3080 NO": [n*0.8/SpeedTest["Stavanger, Norway"] for n in datasetSize],
                "RTX 3080 IS": [n*0.8/SpeedTest["Iceland"] for n in datasetSize],
                "V100 IN": [n*0.8/SpeedTest["India"] for n in datasetSize]
                }


            # compute time
            T3090 = [round(i/2.23, 2) for i in time]
            compt_d = {"Tesla T4 (gpu-e)": time,
                # "Tesla T4 (gpu-f)": time,
                # "RTX 3090 (TY)": T3090,
                "RTX 3090 (UiS)": T3090,
                "A100 80GB": [round(i*1.55, 2) for i in T3090], 
                "A100 40GB": [round(i*1.55, 2) for i in T3090], 
                "V100 IL": [round(i*1.62, 2) for i in T3090],
                "RTX A6000 IL": [round(i*0.90, 2) for i in T3090],
                "RTX A6000 NY": [round(i*0.90, 2) for i in T3090],
                "RTX A5000 IL": [round(i*1.22, 2) for i in T3090],
                "RTX A5000 NV": [round(i*1.22, 2) for i in T3090],
                "RTX A5000 NY": [round(i*1.22, 2) for i in T3090],
                "RTX A4000 NY": [round(i*1.57, 2) for i in T3090],
                "RTX A4000 IL": [round(i*1.57, 2) for i in T3090],
                "A40 IL": [round(i*0.93, 2) for i in T3090], 
                "A40 NY": [round(i*0.93, 2) for i in T3090], 
                "A40 NV": [round(i*0.93, 2) for i in T3090], 
                "QUADRO RTX 5000 IL": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 5000 NY": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 5000 NV": [round(i*1.78, 2) for i in T3090], 
                "QUADRO RTX 4000 IL": [round(i*2.38, 2) for i in T3090],
                "QUADRO RTX 4000 NY": [round(i*2.38, 2) for i in T3090],
                "QUADRO RTX 4000 NV": [round(i*2.38, 2) for i in T3090],
                "RTX 3060 Ti NO": [round(i*1.74, 2) for i in T3090],
                "RTX 3090 NO": T3090,
                "RTX 3090 IS": T3090,
                "RTX 3080 NO": [round(i*1.14, 2) for i in T3090],
                "RTX 3080 IS": [round(i*1.14, 2) for i in T3090],
                "V100 IN": [round(i*1.62, 2) for i in T3090]
                }

            # tot_time = compt_time+trans_time
            computeDf = pd.DataFrame(data=compt_d)
            transDF = pd.DataFrame(data=trans_time)

            tottDf = computeDf+transDF
            costDf = tottDf * UP
            print(tottDf)


            
            time_matrix = tottDf.to_numpy()
            cost_matrix = costDf.to_numpy()
            print(costDf)
            

            n, m = time_matrix.shape  # n行m列
            print(n, m)

            # 定义最大总成本ε
            # max_total_cost = 100  # 这个值根据实际情况调整
            # max_total_cost = 120  # 这个值根据实际情况调整
            max_total_cost = 80  # 这个值根据实际情况调整

            # 创建问题实例
            prob = pulp.LpProblem("MinimizeMaxTimeUnderCostConstraint", pulp.LpMinimize)

            # 定义决策变量
            x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n) for j in range(m)), cat='Binary')
            # 定义一个辅助变量来表示最大完成时间
            max_time = pulp.LpVariable("max_time", lowBound=0)

            # 目标函数：最小化最大完成时间
            prob += max_time

            # 约束条件
            # 每行恰好选择一个元素
            for i in range(n):
                prob += pulp.lpSum(x[(i, j)] for j in range(m)) == 1

            # 每列最多选择一个元素
            for j in range(m):
                prob += pulp.lpSum(x[(i, j)] for i in range(n)) <= 1

            # 总成本不超过ε
            prob += pulp.lpSum(x[(i, j)] * cost_matrix[i, j] for i in range(n) for j in range(m)) <= max_total_cost

            # 最大完成时间约束
            for i in range(n):
                for j in range(m):
                    prob += x[(i, j)] * time_matrix[i, j] <= max_time

            # 求解问题
            prob.solve()

            # 检查是否有最优解
            if pulp.LpStatus[prob.status] == "Optimal":
                solution = [j for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) > 0.5]
                total_cost = sum(cost_matrix[i, solution[i]] for i in range(n))
                max_time_value = pulp.value(max_time)
                assignment = [(i, j) for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) == 1]
                print(f"Solution: {solution}, Total Cost: {total_cost}, Max Time: {max_time_value}, Assignment: {assignment}")
                solution_per_strategy = {'Total Cost': total_cost,
                                    'Max Time': max_time_value,
                                    'Solution': solution,
                                    'assignment': assignment }
                Even_solutions4strategies[(my_strategy, dataset_number)] = solution_per_strategy
            else:
                print("No optimal solution found.")
                continue 

            print(my_strategy) 
            
            # # 打印解决方案
            # for solution in solutions:
            #     print(f"当最大时间限制为 {solution['epsilon']} 时，最小总成本为 {solution['total_cost']}, 分配方案为 {solution['assignment']}")
            # # print(solutions)

is_shuffled_True 4 [25.505, 25.505, 25.5, 25.5]
[5101, 5101, 5100, 5100]
   Tesla T4 (gpu-e)  RTX 3090 (UiS)   A100 80GB   A100 40GB     V100 IL  RTX A6000 IL  RTX A6000 NY  RTX A5000 IL  RTX A5000 NV  RTX A5000 NY  RTX A4000 NY  RTX A4000 IL      A40 IL      A40 NY      A40 NV  QUADRO RTX 5000 IL  QUADRO RTX 5000 NY  QUADRO RTX 5000 NV  QUADRO RTX 4000 IL  QUADRO RTX 4000 NY  QUADRO RTX 4000 NV  RTX 3060 Ti NO  RTX 3090 NO  RTX 3090 IS  RTX 3080 NO  RTX 3080 IS    V100 IN
0        126.866153       54.422937  201.384365  201.384365  202.184365    193.954365    148.351421    197.614365    145.386731    152.011421    156.011421    201.614365  194.294365  148.691421  142.066731          204.014365          158.411421          151.786731          210.884365          165.281421          158.656731       62.892937    54.422937    71.762247    56.022937    73.362247  65.953591
1        126.866153       54.422937  201.384365  201.384365  202.184365    193.954365    148.351421    197.614365    

In [22]:
print(Even_solutions4strategies)
# print(Even_solutions4strategies.keys())
# print(Even_solutions4strategies.values())

{('is_shuffled_True', 4): {'Total Cost': 49.39506673688646, 'Max Time': 126.84128, 'Solution': [24, 21, 0, 1], 'assignment': [(0, 24), (1, 21), (2, 0), (3, 1)]}, ('is_shuffled_True', 5): {'Total Cost': 65.19411124921004, 'Max Time': 101.47303, 'Solution': [1, 22, 0, 24, 21], 'assignment': [(0, 1), (1, 22), (2, 0), (3, 24), (4, 21)]}, ('is_shuffled_True', 6): {'Total Cost': 78.77761962193105, 'Max Time': 84.560854, 'Solution': [1, 25, 22, 0, 24, 21], 'assignment': [(0, 1), (1, 25), (2, 22), (3, 0), (4, 24), (5, 21)]}, ('is_shuffled_False', 4): {'Total Cost': 49.39506673688646, 'Max Time': 126.84128, 'Solution': [24, 21, 0, 1], 'assignment': [(0, 24), (1, 21), (2, 0), (3, 1)]}, ('is_shuffled_False', 5): {'Total Cost': 65.19411124921004, 'Max Time': 101.47303, 'Solution': [1, 22, 0, 24, 21], 'assignment': [(0, 1), (1, 22), (2, 0), (3, 24), (4, 21)]}, ('is_shuffled_False', 6): {'Total Cost': 78.77761962193105, 'Max Time': 84.560854, 'Solution': [1, 25, 22, 0, 24, 21], 'assignment': [(0, 1)

# Solution 2

In [ ]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import pulp


def kLargest(arr, k, flag):
	# Sort the given array arr in reverse order.
	arr.sort(reverse=flag)
	# Print the first kth largest elements
	for i in range(k):
		print(arr[i], end=" ")


# time cost of using CNN workloads: q*F(No. of patches) = latencies; --> latencies/F(No.of patches) --> time_cost_base: q second per patch 
time_base = 0.0052 

for new_file_path in new_file_paths: 
    graph_split_file = f'{new_file_path}/graph_split1/'
    os.listdir(graph_split_file)
    all_solutions = {}
    strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 'connected_sequential', 'saturation_largest_first']

    solutions4strategies = {}
    
    for my_strategy in strategies: 
        # 设置你的目标文件夹名称和文件模式
        graph_split_file = f'{new_file_path}/graph_split1/'
        file_pattern = f"*[0-9]{my_strategy}.csv"

        # 计算文件数量
        dataset_number = count_files_with_numeric_pattern(graph_split_file, file_pattern)

        tmp = []
        for j in range(dataset_number):
            # sub data set
            origfpath = f'{graph_split_file}/graph_patchesnames_{str(j)}{my_strategy}.csv'
            df = pd.read_csv(origfpath)
            data_no = len(df['PatchName'])
            time4data_no = data_no * time_base
            tmp.append(time4data_no)
        # print(my_strategy, tmp)
        time_per_strategy[my_strategy] = tmp

        time = tmp

        T3090 = [round(i/2.23, 2) for i in time]
        relativePerf = {"T4": 1,
                        "3090": 2.23, 
                        }
        d = {"Tesla T4 (gpu-e)": time,
            #  "Tesla T4 (gpu-f)": time,
            #  "RTX 3090 (TY)": T3090,
            "RTX 3090 (UiS)": T3090,
            "A100 80GB": [round(i*1.55, 2) for i in T3090], 
            "A100 40GB": [round(i*1.55, 2) for i in T3090], 
            "V100 IL": [round(i*1.62, 2) for i in T3090],
            "RTX A6000 IL": [round(i*0.90, 2) for i in T3090],
            "RTX A6000 NY": [round(i*0.90, 2) for i in T3090],
            "RTX A5000 IL": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NV": [round(i*1.22, 2) for i in T3090],
            "RTX A5000 NY": [round(i*1.22, 2) for i in T3090],
            "RTX A4000 NY": [round(i*1.57, 2) for i in T3090],
            "RTX A4000 IL": [round(i*1.57, 2) for i in T3090],
            "A40 IL": [round(i*0.93, 2) for i in T3090], 
            "A40 NY": [round(i*0.93, 2) for i in T3090], 
            "A40 NV": [round(i*0.93, 2) for i in T3090], 
            "QUADRO RTX 5000 IL": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NY": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 5000 NV": [round(i*1.78, 2) for i in T3090], 
            "QUADRO RTX 4000 IL": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NY": [round(i*2.38, 2) for i in T3090],
            "QUADRO RTX 4000 NV": [round(i*2.38, 2) for i in T3090],
            "RTX 3060 Ti": [round(i*1.74, 2) for i in T3090],
            "RTX 3090 NO": T3090,
            "RTX 3090 IS": T3090,
            "RTX 3080 NO": [round(i*1.14, 2) for i in T3090],
            "RTX 3080 IS": [round(i*1.14, 2) for i in T3090],
            "V100 IN": [round(i*1.62, 2) for i in T3090]
            }

        UP = {"Tesla T4 (gpu-e)": 0,
            #  "Tesla T4 (gpu-f)": 0,
            #  "RTX 3090 (TY)": 0,
            "RTX 3090 (UiS)": 0,
            "A100 80GB": 2.15, 
            "A100 40GB": 2.01, 
            "V100 IL": 0.81,
            "RTX A6000 IL": 1.27,
            "RTX A6000 NY": 1.27,
            "RTX A5000 IL": 0.78,
            "RTX A5000 NV": 0.78,
            "RTX A5000 NY": 0.78,
            "RTX A4000 NY": 0.63,
            "RTX A4000 IL": 0.63,
            "A40 IL": 1.27, 
            "A40 NY": 1.27, 
            "A40 NV": 1.27, 
            "QUADRO RTX 5000 IL": 0.59, 
            "QUADRO RTX 5000 NY": 0.59, 
            "QUADRO RTX 5000 NV": 0.59, 
            "QUADRO RTX 4000 IL": 0.28,
            "QUADRO RTX 4000 NY": 0.28,
            "QUADRO RTX 4000 NV": 0.28,
            "RTX 3060 Ti": 0.34,
            "RTX 3090 NO": 0.59,
            "RTX 3090 IS": 0.59,
            "RTX 3080 NO": 0.50,
            "RTX 3080 IS": 0.50,
            "V100 IN": 1.07
            }

        df2 = pd.DataFrame(data=d)
        time = pd.DataFrame(data=d)
        cost = df2 * UP

        df1 = df2 * UP
        # print(df1)
        time_matrix = df2.to_numpy()
        cost_matrix = df1.to_numpy()

        n, m = time_matrix.shape  # n行m列

        # 定义最大总成本ε
        max_total_cost = 12  # 这个值根据实际情况调整

        # 创建问题实例
        prob = pulp.LpProblem("MinimizeMaxTimeUnderCostConstraint", pulp.LpMinimize)

        # 定义决策变量
        x = pulp.LpVariable.dicts("x", ((i, j) for i in range(n) for j in range(m)), cat='Binary')
        # 定义一个辅助变量来表示最大完成时间
        max_time = pulp.LpVariable("max_time", lowBound=0)

        # 目标函数：最小化最大完成时间
        prob += max_time

        # 约束条件
        # 每行恰好选择一个元素
        for i in range(n):
            prob += pulp.lpSum(x[(i, j)] for j in range(m)) == 1

        # 每列最多选择一个元素
        for j in range(m):
            prob += pulp.lpSum(x[(i, j)] for i in range(n)) <= 1

        # 总成本不超过ε
        prob += pulp.lpSum(x[(i, j)] * cost_matrix[i, j] for i in range(n) for j in range(m)) <= max_total_cost

        # 最大完成时间约束
        for i in range(n):
            for j in range(m):
                prob += x[(i, j)] * time_matrix[i, j] <= max_time

        # 求解问题
        prob.solve()

        # 检查是否有最优解
        if pulp.LpStatus[prob.status] == "Optimal":
            solution = [j for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) > 0.5]
            total_cost = sum(cost_matrix[i, solution[i]] for i in range(n))
            max_time_value = pulp.value(max_time)
            assignment = [(i, j) for i in range(n) for j in range(m) if pulp.value(x[(i, j)]) == 1]
            print(f"Solution: {solution}, Total Cost: {total_cost}, Max Time: {max_time_value}, Assignment: {assignment}")
        else:
            print("No optimal solution found.")

        print(my_strategy) 
        solution_per_strategy = {'Total Cost': total_cost,
                                'Max Time': max_time_value,
                                'Solution': solution,
                                'assignment': assignment }
        solutions4strategies[my_strategy] = solution_per_strategy
        # # 打印解决方案
        # for solution in solutions:
        #     print(f"当最大时间限制为 {solution['epsilon']} 时，最小总成本为 {solution['total_cost']}, 分配方案为 {solution['assignment']}")
        # # print(solutions)
   

In [ ]:
print(solutions4strategies)

In [ ]:
# 1WSI
privacyscores = {'largest_first': (0.11, 0.088), 'random_sequential': (0.111, 0.088), 'smallest_last': (0.152, 0.116), 'independent_set': (0.111, 0.086), 'connected_sequential': (0.127, 0.098), 'saturation_largest_first': (0.183, 0.139)}
solutions = {'largest_first': {'Total Cost': 11.889, 'Max Time': 7.73, 'Solution': [25, 23, 22, 1], 'assignment': [(0, 25), (1, 23), (2, 22), (3, 1)]}, 'random_sequential': {'Total Cost': 11.889, 'Max Time': 7.73, 'Solution': [25, 23, 22, 1], 'assignment': [(0, 25), (1, 23), (2, 22), (3, 1)]}, 'smallest_last': {'Total Cost': 11.889, 'Max Time': 7.73, 'Solution': [25, 23, 22, 1], 'assignment': [(0, 25), (1, 23), (2, 22), (3, 1)]}, 'independent_set': {'Total Cost': 11.889, 'Max Time': 7.73, 'Solution': [25, 23, 22, 1], 'assignment': [(0, 25), (1, 23), (2, 22), (3, 1)]}, 'connected_sequential': {'Total Cost': 11.889, 'Max Time': 7.73, 'Solution': [25, 23, 22, 1], 'assignment': [(0, 25), (1, 23), (2, 22), (3, 1)]}, 'saturation_largest_first': {'Total Cost': 11.889, 'Max Time': 7.73, 'Solution': [25, 23, 22, 1], 'assignment': [(0, 25), (1, 23), (2, 22), (3, 1)]}}



# 4WSI
# privacyscores = {'largest_first': (0.112, 0.095), 'random_sequential': (0.112, 0.096), 'smallest_last': (0.156, 0.13), 'independent_set': (0.112, 0.093), 'connected_sequential': (0.128, 0.107), 'saturation_largest_first': (0.185, 0.152)}
# solutions = {'largest_first': {'Total Cost': 11.684499999999998, 'Max Time': 5.55, 'Solution': [1, 23, 22, 24, 25, 0, 5, 19], 'assignment': [(0, 1), (1, 23), (2, 22), (3, 24), (4, 25), (5, 0), (6, 5), (7, 19)]}, 'random_sequential': {'Total Cost': 11.6935, 'Max Time': 5.5692, 'Solution': [22, 1, 23, 25, 24, 0, 5, 19], 'assignment': [(0, 22), (1, 1), (2, 23), (3, 25), (4, 24), (5, 0), (6, 5), (7, 19)]}, 'smallest_last': {'Total Cost': 11.3712, 'Max Time': 7.19, 'Solution': [22, 1, 23, 24, 0], 'assignment': [(0, 22), (1, 1), (2, 23), (3, 24), (4, 0)]}, 'independent_set': {'Total Cost': 11.877699999999999, 'Max Time': 7.12, 'Solution': [23, 1, 22, 24, 25, 0, 20], 'assignment': [(0, 23), (1, 1), (2, 22), (3, 24), (4, 25), (5, 0), (6, 20)]}, 'connected_sequential': {'Total Cost': 11.657499999999999, 'Max Time': 7.4, 'Solution': [23, 22, 1, 25, 0, 21], 'assignment': [(0, 23), (1, 22), (2, 1), (3, 25), (4, 0), (5, 21)]}, 'saturation_largest_first': {'Total Cost': 11.889, 'Max Time': 7.73, 'Solution': [25, 23, 22, 1], 'assignment': [(0, 25), (1, 23), (2, 22), (3, 1)]}}

strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 'connected_sequential', 'saturation_largest_first']

input1 = [round(privacyscores[s][0] + privacyscores[s][1], 3) for s in strategies]
input2 = [round(solutions[s]['Total Cost'], 3) for s in strategies]
input3 = [round(solutions[s]['Max Time'], 3) for s in strategies]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt


def simple_cull(inputPoints, dominates, sMax=True):
    paretoPoints = set()
    candidateRowNr = 0
    dominatedPoints = set()
    while True:
        candidateRow = inputPoints[candidateRowNr]
        inputPoints.remove(candidateRow)
        rowNr = 0
        nonDominated = True
        while len(inputPoints) != 0 and rowNr < len(inputPoints):
            row = inputPoints[rowNr]
            
            if dominates(candidateRow, row, sMax):
                # If it is worse on all features remove the row from the array
                inputPoints.remove(row)
                dominatedPoints.add(tuple(row))
            elif dominates(row, candidateRow, sMax):
                nonDominated = False
                dominatedPoints.add(tuple(candidateRow))
                rowNr += 1
            else:
                rowNr += 1

        if nonDominated:
            # add the non-dominated point to the Pareto frontier
            paretoPoints.add(tuple(candidateRow))

        if len(inputPoints) == 0:
            break
    return paretoPoints, dominatedPoints

def dominates(row, candidateRow, sMax = True):
    if sMax:
        return sum([row[x] >= candidateRow[x] for x in range(len(row))]) == len(row)  
    else: 
        return sum([row[x] <= candidateRow[x] for x in range(len(row))]) == len(row)


In [ ]:
from functools import reduce
import re
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
from pareto_3D import simple_cull, dominates


def has_unique_elements(input_list):
    return len(input_list) == len(set(input_list))

def remove_duplicates(lst):
    unique_set = set()
    unique_list = []
    for inner_list in lst:
        # Convert the inner list to a tuple to make it hashable
        tuple_inner_list = tuple(inner_list)
        if tuple_inner_list not in unique_set:
            unique_list.append(inner_list)
            unique_set.add(tuple_inner_list)
    return unique_list

def mapping2time(input_list, a):
    result_dict = {}
    for i, item in enumerate(input_list):
        if item in result_dict:
            result_dict[item].append(a[i])
        else:
            result_dict[item] = [a[i]]
    return result_dict

# write list to txt
def savelist2txt(l, destfpath):
    # assign list
    # l = ['Geeks','for','Geeks!']
    # open file
    with open(destfpath, 'w+') as f:
        
        # write elements of list
        for items in l:
            f.write('%s\n' %items)
        
        print("File written successfully")

    # close the file
    f.close()

# read txt to list
def readtxt2list(orgfpath):
    l = []
    with open(orgfpath, 'r+') as f:
        
        # read elements of txt
        for data in f.readlines():
            t1 = data.strip()
            t2 = t1.split('[')[1]
            t3 = t2.split(']')[0]
            t4 = t3.split(', ')
            x = [float(i) for i in t4]
            # print(x)
            l.append(x)
        # print("File read successfully")
    # close the file
    f.close()
    return l

# Define a custom formatting function
# def z_formatter(x, pos):
#     return f"{x*10000:.0f}"

# def y_formatter(x, pos):
#     return f"{x/10000:.0f}"

fn2 = lambda x, code='': reduce(lambda x, y: [str(i)+code+str(j) for i in x for j in y], x)


print("HAHA")

# 1WSI
privacyscores = {'largest_first': (0.11, 0.088), 'random_sequential': (0.111, 0.088), 'smallest_last': (0.152, 0.116), 'independent_set': (0.111, 0.086), 'connected_sequential': (0.127, 0.098), 'saturation_largest_first': (0.183, 0.139)}
solutions = {'largest_first': {'Total Cost': 97.3211014671732, 'Max Time': 45.207942, 'Solution': [1, 22, 24, 21, 25, 23, 0, 5], 'assignment': [(0, 1), (1, 22), (2, 24), (3, 21), (4, 25), (5, 23), (6, 0), (7, 5)]}, 'random_sequential': {'Total Cost': 105.19388040434416, 'Max Time': 43.649366, 'Solution': [1, 22, 24, 21, 25, 23, 16, 0], 'assignment': [(0, 1), (1, 22), (2, 24), (3, 21), (4, 25), (5, 23), (6, 16), (7, 0)]}, 'smallest_last': {'Total Cost': 79.81389483884558, 'Max Time': 60.541925, 'Solution': [22, 1, 24, 21, 0], 'assignment': [(0, 22), (1, 1), (2, 24), (3, 21), (4, 0)]}, 'independent_set': {'Total Cost': 84.32925510454895, 'Max Time': 57.4032, 'Solution': [22, 24, 1, 21, 0, 6, 25], 'assignment': [(0, 22), (1, 24), (2, 1), (3, 21), (4, 0), (5, 6), (6, 25)]}, 'connected_sequential': {'Total Cost': 82.83910125733902, 'Max Time': 59.860048, 'Solution': [1, 22, 24, 21, 25, 0], 'assignment': [(0, 1), (1, 22), (2, 24), (3, 21), (4, 25), (5, 0)]}, 'saturation_largest_first': {'Total Cost': 81.47333084053088, 'Max Time': 62.802378, 'Solution': [1, 24, 22, 21], 'assignment': [(0, 1), (1, 24), (2, 22), (3, 21)]}}

# 4WSI
# privacyscores = {'largest_first': (0.112, 0.095), 'random_sequential': (0.112, 0.096), 'smallest_last': (0.156, 0.13), 'independent_set': (0.112, 0.093), 'connected_sequential': (0.128, 0.107), 'saturation_largest_first': (0.185, 0.152)}
# solutions = {'largest_first': {'Total Cost': 77.68596648549763, 'Max Time': 25.742696, 'Solution': [1, 22, 24, 21, 26, 23, 5, 0], 'assignment': [(0, 1), (1, 22), (2, 24), (3, 21), (4, 26), (5, 23), (6, 5), (7, 0)]}, 'random_sequential': {'Total Cost': 79.13511034307052, 'Max Time': 25.3362, 'Solution': [22, 1, 24, 21, 26, 23, 5, 0], 'assignment': [(0, 22), (1, 1), (2, 24), (3, 21), (4, 26), (5, 23), (6, 5), (7, 0)]}, 'smallest_last': {'Total Cost': 44.39158653886665, 'Max Time': 33.378622, 'Solution': [1, 24, 22, 21, 0], 'assignment': [(0, 1), (1, 24), (2, 22), (3, 21), (4, 0)]}, 'independent_set': {'Total Cost': 45.242576410730415, 'Max Time': 33.034257, 'Solution': [22, 1, 24, 21, 0, 23, 9], 'assignment': [(0, 22), (1, 1), (2, 24), (3, 21), (4, 0), (5, 23), (6, 9)]}, 'connected_sequential': {'Total Cost': 45.41958705990994, 'Max Time': 34.327453, 'Solution': [22, 1, 24, 21, 0, 25], 'assignment': [(0, 22), (1, 1), (2, 24), (3, 21), (4, 0), (5, 25)]}, 'saturation_largest_first': {'Total Cost': 46.60948487465768, 'Max Time': 35.852324, 'Solution': [21, 1, 24, 22], 'assignment': [(0, 21), (1, 1), (2, 24), (3, 22)]}}

strategies = ['largest_first', 'random_sequential', 'smallest_last', 'independent_set', 'connected_sequential', 'saturation_largest_first']

input1 = [round(privacyscores[s][0] + privacyscores[s][1], 3) for s in strategies]
input2 = [round(solutions[s]['Total Cost'], 3) for s in strategies]
input3 = [round(solutions[s]['Max Time'], 3) for s in strategies]
inputPoints = [[input1[j], input2[j], input3[j] ] for j in range(len(input1))]

# paretoPoints, dominatedPoints = simple_cull(inputPoints, dominates, sMax=True) # find maximum
paretoPoints, dominatedPoints = simple_cull(inputPoints, dominates, sMax=False) # find minimum
print("paretoPoints: \n", paretoPoints, )
print("dominatedPoints: \n", dominatedPoints)

dp = np.array(list(dominatedPoints))
pp = np.array(list(paretoPoints))
print(pp.shape, dp.shape, len(set(list(pp[:,0]))))
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['text.usetex'] = True


if len(set(list(pp[:,0])))==1:
    fig = plt.figure(layout="constrained")
    # (ax1, ax2) = fig.subplots(2, 2, squeeze=False)
    # fig, axs = plt.subplots(2, 1, subplot_kw={'projection': '3d'})

    # axs = fig.add_subplot(111, projection='3d')
    axs = plt.axes(projection='3d')
    
    print("lala", paretoPoints)
    # axs.scatter(dp[:,0],dp[:,1],dp[:,2], color='darkcyan', label='None Pareto optimal', alpha = 0.6)

    axs.scatter(dp[:,0],dp[:,1],dp[:,2], label='None Pareto optimal', alpha = 0.6, s=80, color='blue')
    axs.scatter(pp[:,0],pp[:,1],pp[:,2], color='red',marker='o', s=80, label="Pareto optimal")
    # axs.scatter(pp[:,0],pp[:,1],pp[:,2], s=80, label="Pareto optimal", alpha = 0.6)
    axs.legend()
    print(pp[:,0])
    xdata = pp[:,0]
    ydata = pp[:,1]
    zdata = pp[:,2]
    
    solution_size = len(list(xdata))
    # axs.text2D(0.05, 0.95, f"First-6 out of {solution_size} Pareto optimal solutions", transform=axs.transAxes)

    # for i in range(len(list(xdata[:6]))):
        # axs.text2D(0.05, 0.9-0.05*i, f'Pfdata[{i}] = ({xdata[i]:.0f}, {ydata[i]:.3f}, {zdata[i]:.6f})', transform=axs.transAxes)

    # Set the formatter for the y-axis tick labels
    # axs.zaxis.set_major_formatter(ticker.FuncFormatter(z_formatter))
    solution_size = len(list(xdata))
    axs.text2D(0.05, 0.95, f"First-6 out of {solution_size} Pareto optimal solutions", transform=axs.transAxes)

    
    for i in range(len(list(xdata[:6]))):
        axs.text2D(0.05, 0.9-0.05*i, f'Pfdata[{i}] = ({xdata[i]:.3f}, {ydata[i]:.3f}, {zdata[i]:.3f})', transform=axs.transAxes)


    # axs.set_title("Pareto frontier 3D")
    axs.set_xlabel(r'Privacy')
    # axs.set_xticks(np.arange(min(dp[:,0])-1, max(dp[:,0])+1,1))
    axs.set_ylabel(r'Total Cost')
    axs.set_zlabel(r'Max Time')
    # plt.savefig('/Volumes/T7/Research/WSI preprocessing project/Distributed Image Processing/image_transmission/benchmarks/res/opt10Data/'+"pareto3D_res_T"+str(Threshold)+"_C"+str(Budget)+"_N" + str(NO)+"_update.png")
    # plt.savefig(resPareto3dpath+f"T"+str(Threshold)+"_C"+str(Budget)+"_N" + str(NO)+"_update.png")
    # plt.savefig(f"./f3"+str(Threshold)+"_f2"+str(Budget)+"_f1" + str(Privacy)+"_update.png")
    plt.show()
else:
    fig = plt.figure(layout="constrained")
    # (ax1, ax2) = fig.subplots(2, 2, squeeze=False)
    # fig, axs = plt.subplots(2, 1, subplot_kw={'projection': '3d'})

    # axs = fig.add_subplot(111, projection='3d')
    axs = plt.axes(projection='3d')

    # axs.scatter(dp[:,0],dp[:,1],dp[:,2], color = 'blue', alpha = 0.6)
    # axs.scatter(pp[:,0], pp[:,1], pp[:,2], color = 'red')
    # plt.show()

    import matplotlib.tri as mtri

    # import matplotlib.pyplot as plt
    # from mpl_toolkits.mplot3d import Axes3D

    print(pp)

    if len(dominatedPoints)>=2 and len(paretoPoints)>=2: 
        # axs.scatter(dp[:,0],dp[:,1],dp[:,2], color='darkcyan', label='None Pareto optimal', alpha = 0.6)
        # axs.scatter(pp[:,0],pp[:,1],pp[:,2], color='red',marker='o', s=80, label="Pareto optimal")
        axs.scatter(dp[:,0],dp[:,1],dp[:,2], label='None Pareto optimal', alpha = 0.6, s=80)
        axs.scatter(pp[:,0],pp[:,1],pp[:,2], color='red',s=80, label="Pareto optimal")
        triang = mtri.Triangulation(pp[:,0], pp[:,1])
        axs.plot_trisurf(triang,pp[:,2], color='azure', alpha = 0.6)
    elif len(paretoPoints)<=2 or len(dominatedPoints)<=2: 
        # axs.scatter(dp[:,0],dp[:,1],dp[:,2], color='darkcyan', label='None Pareto optimal', alpha = 0.6)
        # axs.scatter(pp[:,0],pp[:,1],pp[:,2], color='red', label="Pareto optimal")
        axs.scatter(dp[:,0],dp[:,1],dp[:,2], label='None Pareto optimal', alpha = 0.6, s=80, color='blue')
        axs.scatter(pp[:,0],pp[:,1],pp[:,2], color='red', s=80, label="Pareto optimal")
    elif len(paretoPoints)>2 and len(dominatedPoints)<=2:
        # axs.scatter(dp[:,0],dp[:,1],dp[:,2], color='darkcyan', label='None Pareto optimal', alpha = 0.6)
        # axs.scatter(pp[:,0],pp[:,1],pp[:,2], color='red', marker='o', s=80, label="Pareto optimal")
        axs.scatter(dp[:,0],dp[:,1],dp[:,2], label='None Pareto optimal', alpha = 0.6, s=80, color='blue')
        axs.scatter(pp[:,0],pp[:,1],pp[:,2], color='red', s=80, label="Pareto optimal")
        triang = mtri.Triangulation(pp[:,0], pp[:,1])
        axs.plot_trisurf(triang,pp[:,2], color='azure', alpha = 0.6)
    else: 
        # axs.scatter(dp[:,0],dp[:,1],dp[:,2], color='darkcyan', label='None Pareto optimal', alpha = 0.6)
        # axs.scatter(pp[:,0],pp[:,1],pp[:,2], color='red', label="Pareto optimal")
        axs.scatter(dp[:,0],dp[:,1],dp[:,2], label='None Pareto optimal', alpha = 0.6, color='blue')
        axs.scatter(pp[:,0],pp[:,1],pp[:,2], color='red', s=80, label="Pareto optimal")
        

    axs.legend()
    print(pp[:,0])
    xdata = pp[:,0]
    ydata = pp[:,1]
    zdata = pp[:,2]
    
    solution_size = len(list(xdata))
    axs.text2D(0.05, 0.95, f"First-6 out of {solution_size} Pareto optimal solutions", transform=axs.transAxes)

    
    for i in range(len(list(xdata[:6]))):
        axs.text2D(0.05, 0.9-0.05*i, f'Pfdata[{i}] = ({xdata[i]:.3f}, {ydata[i]:.3f}, {zdata[i]:.3f})', transform=axs.transAxes)

    # Set the formatter for the y-axis tick labels
    # axs.zaxis.set_major_formatter(ticker.FuncFormatter(z_formatter))
    # axs.yaxis.set_major_formatter(ticker.FuncFormatter(y_formatter))

    # axs.set_title("Pareto frontier 3D")
    axs.set_xlabel(r'Privacy')
    # axs.set_xticks(np.arange(min(dp[:,0]), max(dp[:,0])+1,1))
    axs.set_ylabel(r'Total Cost')
    axs.set_zlabel(r'Max Time')
    # plt.savefig('/Volumes/T7/Research/WSI preprocessing project/Distributed Image Processing/image_transmission/benchmarks/res/opt10Data/'+"pareto3D_res_T"+str(Threshold)+"_C"+str(Budget)+"_N" + str(NO)+"_update.png")
    plt.savefig(f"./f3"+"_f2"+"_f1_update.png")
    plt.show()
